#### imports

In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet

In [ ]:
df = pd.read_excel('facts 2022-2023.xlsx')
df

#### cleaning data

In [ ]:
df['Кафе'].unique()

In [4]:
df = df[df['Кафе']!='Кейтеринг']

In [ ]:
df.isnull().sum()

In [6]:
df = df[['Кафе', 'Дата', 'Тр', 'Чек']]

In [ ]:
df.info()

#### forecasting

In [8]:
holiday = pd.read_excel('holidays_df.xlsx')

In [9]:
def forec_func(obj, result_df):
    cafes = df['Кафе'].unique().tolist()

    for cafe in cafes:
        df_cafe = df[df['Кафе'] == cafe]
        df_cafe = df_cafe[['Дата', obj]]
        df_cafe.columns = ['ds', 'y']

        # Создание нового экземпляра модели для каждого кафе
        m = Prophet(holidays=holiday)
        m.add_country_holidays(country_name='RU')

        # Обучение модели на данных каждого кафе
        m.fit(df_cafe)

        # Создание будущего датафрейма и выполнение прогноза
        future = m.make_future_dataframe(periods=365)
        forecast = m.predict(future)
        
        forecast["Кафе"] = cafe
        forecast["Показатель"] = obj

        # Сохранение прогноза в result_df
        result_df = pd.concat([result_df, forecast])
    return result_df

In [10]:
result_df = pd.DataFrame()

In [ ]:
result_df = forec_func('Тр', result_df)

In [ ]:
result_df = forec_func('Чек', result_df)

#### Prepare result and save to Excel

In [ ]:
result_df.info()

In [44]:
pivot_df = result_df.set_index('ds')

In [46]:
pivot_df = pivot_df.pivot_table(index=['ds', 'Кафе'], columns='Показатель', values='yhat').reset_index()

In [ ]:
pivot_df

In [48]:
pivot_df['ТО'] = pivot_df['Тр'] * pivot_df['Чек']

In [51]:
pivot_df.to_excel('forecast_revenue.xlsx', index=False)